## Importaciones

In [1]:
import pandas as pd
import math
import numpy as np
from random import randint
import hashlib
import re

from tqdm import tqdm

## Hiperparámetros

In [53]:
signature_size = 50     # Numero de funciones hash
bands_nr = 10           # Numero de bandas
threshold = 0.5         # Umbral de similitud
upper_threshold = 0.65   # Umbral de similitud superior
user_thereshold = 4     # Umbral de usuarios similares
k = 3                   # Largo de los shingles

total_tweets = 0.1      # Porcentaje de tweets a procesar

## Cargar datos y sacar muestra

In [3]:
req_cols = ['id','screen_name','text']
row_num = math.ceil(total_tweets * 4594980)  # % de los datos

# Abrir el archivo CSV y procesarlo línea por línea
tweets_df = pd.read_csv("tweets_2022_abril_junio.csv", usecols=req_cols, nrows=row_num)

tweets_df.columns

Index(['id', 'screen_name', 'text'], dtype='object')

In [4]:
doc_nr = len(tweets_df)
doc_nr

459498

## Preprocesamiento de datos

In [5]:
for index, row in tqdm(tweets_df.iterrows()):
    text = row['text'].replace('\n', '').replace('\r', '')
    text = re.sub(r'^RT\s+@\w+:\s+', '',text).lower()
    text = re.sub(r'@\w+', '', text)
    text = text.lstrip(' ').rstrip(' ')
    tweets_df.at[index, 'text'] = text

459498it [00:41, 11095.26it/s]


In [6]:
tweets_df.head()

,id,screen_name,text
0,1512186166438637582,h0l4d4ni3l4,"tras casi 50 años del golpe, la constitución s..."
1,1512186202367045642,Claudio70932894,mañana jueves a las 18hrs. comienza nuestro pr...
2,1512186287284924418,Cesar_A_RR,aquí está el aporte de con respecto a los der...
3,1512186335754301446,rosmarieher,la pelotudez no tiene limites...no tiene
4,1512186407841767424,GQuelluen,"ante la circulación de noticias falsas, les qu..."


## Obtener Shingles por tweet y todos los shingles

In [7]:
tweets_df["shingles"] = [set([tweet[i:i+k] for i in range(len(tweet) - k + 1)]) for tweet in tqdm(tweets_df["text"])]

100%|██████████| 459498/459498 [00:15<00:00, 29903.57it/s]


## Funciones de Hash

In [8]:
class hashFamily:
    def __init__(self, i):
        self.resultSize = 8 # Cantidad de bytes que queremos de vuelta
        self.maxLen = # largo de nuestro i (en decimal)
        self.salt = str(i).zfill(self.maxLen)[-self.maxLen:]
        
    def get_hash_value(self, el_to_hash):
        return int(hashlib.sha1(str(el_to_hash).encode('utf-8') + self.salt.encode('utf-8')).hexdigest()[-self.resultSize:], 16)

## Calculamos el minhash de los tweets

In [9]:
class minhashSigner:
    def __init__(self, sig_size):
        self.sig_size=sig_size
        self.hash_functions = [hashFamily(randint(0,10000000000)) for i in range(0,sig_size)]
    
    def compute_set_signature(self, set_):
        set_sig = []
        for h_funct in self.hash_functions:
            min_hash = math.inf
            for el in set_:
                h = h_funct.get_hash_value(el)
                if h < min_hash:
                    min_hash = h
                
            set_sig.append(min_hash)
        
        return set_sig
    
    # retorna una lista de listas que representa la matriz de firmas
    def compute_signature_matrix(self, set_list):
        signatures = []
        for s in tqdm(set_list):
            signatures.append( self.compute_set_signature(s) )
            
        return signatures

In [10]:
shingling_list = tweets_df["shingles"]
signer = minhashSigner(signature_size)
signature_matrix = signer.compute_signature_matrix( shingling_list )

100%|██████████| 459498/459498 [41:31<00:00, 184.46it/s] 


## Almacenamos los minhash para ahorrar tiempo

In [11]:
np.savetxt('minhash.txt', signature_matrix)

In [12]:
signature_matrix = np.loadtxt('minhash.txt')

In [13]:
signature_matrix

array([[3.06944000e+06, 6.15750800e+06, 1.23405820e+07, ...,
        1.71719110e+07, 3.27035330e+07, 1.08962560e+07],
       [2.12782870e+07, 9.19297970e+07, 1.50364100e+06, ...,
        1.10945680e+08, 4.05155430e+07, 1.81703540e+07],
       [3.06944000e+06, 6.15750800e+06, 1.50364100e+06, ...,
        1.71719110e+07, 2.25454490e+07, 1.12544487e+08],
       ...,
       [3.10876220e+07, 7.13054440e+07, 1.50333113e+08, ...,
        9.44738490e+07, 3.51201770e+07, 5.03140310e+07],
       [3.10876220e+07, 7.13054440e+07, 1.50333113e+08, ...,
        9.44738490e+07, 3.51201770e+07, 5.03140310e+07],
       [3.06944000e+06, 6.15750800e+06, 1.50364100e+06, ...,
        1.04523956e+08, 2.25454490e+07, 3.55349100e+06]])

## LSH (Locality Sensitive Hashing)

In [52]:
class lsh:
    def __init__(self, threshold=0.8):
        self.threshold = threshold
        
    def get_signature_matrix_bands(self, sig_matrix, bands_nr, sign_len): 
        #bands_nr = b
        #sign_len = n
        r = int(sign_len/bands_nr) #numero de filas en cada banda
        bands = {} # {band_nr: [col_1,col_2,...]} donde col_1 son todos los valores de Sig(S_i) para la banda b.
        for i in range(0,bands_nr):
            bands[i] = []
        
        # pone subconjuntos de las columnas de la matriz de firma en el bucket apropiado y cosidera una columna
        # como un bloque único para que podamos hashear toda la columna.
        # Básicamente, una banda es una lista de elementos, donde cada elemento es un subconjunto de una firma de un conjunto dado.
        for signature in tqdm(sig_matrix): 
            
            for i in range(0, bands_nr):
                idx = i*r    
                bands[i].append(' '.join(str(x) for x in signature[idx:idx+r]) ) 
                    
        return bands

    #band is a list 
    # contruye un diccionario {hash(band_column): doc_id que produjo este hash}
    def get_band_buckets(self, band, hash_funct):
        buckets = {}
        for doc_id in tqdm(range(0,len(band))):
            value = hash_funct.get_hash_value( band[doc_id] )
            if value not in buckets:
                buckets[value] = [doc_id]
            else:
                 buckets[value].append(doc_id)
                
        return buckets
    
    def get_candidates_list(self, buckets):
        candidates = set()
        # buckets es un diccionario que contiene key=bucket, value= lista de doc_ids que se hashearon a bucket
        for bucket,candidate_list in tqdm(buckets.items()):
            if len(candidate_list) > 1:
                for i in range(0,len(candidate_list)-1):
                    for j in range(i+1,len(candidate_list)):  
                        pair = tuple(sorted( (candidate_list[i],candidate_list[j]) ))
                        candidates.add(pair)
                
        return candidates # un conjunto de tuplas, cada tupla es un par candidato
    
    def check_candidates(self, candidates_list, threshold, sigs):
        similar_docs = set() #set de tuplas
        # similar_pair es una pareja que contiene doc_ids de documentos que se hashearon al mismo bucket
        for  similar_pair in tqdm(candidates_list):
            # para todos los pares de documentos en la lista, verifica la similitud de sus firmas
            doc_id_1 = similar_pair[0]
            doc_id_2 = similar_pair[1]
            signature_1 = set(sigs[doc_id_1]) # obtiene la columna i de la matriz de firma donde i es doc_id en la lista de colisiones
            signature_2 = set(sigs[doc_id_2])
            js = len(signature_1.intersection(signature_2)) /len(signature_1.union(signature_2))
            
            if js >= threshold and js < upper_threshold:
                similar_docs.add( tuple(sorted((doc_id_1,doc_id_2) )) )
                        
                        
        return similar_docs
    
    def get_similar_items(self, sig_matrix, bands_nr, sign_len):
        similar_docs = set()
        # divide la matriz en firma en bandas
        bands = self.get_signature_matrix_bands(sig_matrix,bands_nr,sign_len)
        
        # para todas las bandas
        for band_id, elements in bands.items():
            print("Band: ", band_id)
            # crea los buckets para la banda dada (band_id) con una función hash aleatoria
            buckets = self.get_band_buckets(elements, hash_funct=hashFamily(randint(0,10000000000)))
            # Obtiene todos los pares candidatos
            candidates = self.get_candidates_list(buckets)
            # Revisa todas las firmas de pares candidatos
            for sim_tuple in self.check_candidates(candidates, self.threshold, sig_matrix):
                similar_docs.add(sim_tuple)

        return similar_docs # Retorna todas las firmas similares que respetan el threshold

In [54]:
lsh_instance = lsh(threshold)
lsh_similar_itemset = lsh_instance.get_similar_items(signature_matrix, bands_nr, signature_size)

np.savetxt('lsh.txt', list(lsh_similar_itemset))

100%|██████████| 459498/459498 [00:16<00:00, 27086.14it/s]


Band:  0


100%|██████████| 46774005/46774005 [11:53<00:00, 65547.54it/s]


Band:  1


100%|██████████| 47948942/47948942 [12:11<00:00, 65573.87it/s]


...

100%|██████████| 46742653/46742653 [11:48<00:00, 66010.58it/s]


Band:  8


100%|██████████| 46798169/46798169 [11:45<00:00, 66324.15it/s]


Band:  9


100%|██████████| 46781851/46781851 [11:45<00:00, 66279.68it/s]


In [56]:
lsh_similar_itemset = np.loadtxt('lsh.txt')

In [57]:
user_candidates = dict()
tweets_candidates = dict()

for i in tqdm(range(len(lsh_similar_itemset))):    
    docs = lsh_similar_itemset[i]
    tweet1_name = tweets_df.iloc[int(docs[0])]["screen_name"]
    tweet2_name = tweets_df.iloc[int(docs[1])]["screen_name"]
    tweet1_text = tweets_df.iloc[int(docs[0])]["text"]
    tweet2_text = tweets_df.iloc[int(docs[1])]["text"]
    names = tuple(sorted((tweet1_name,tweet2_name)))
    if tweet1_name != tweet2_name:
        if names not in user_candidates.keys():
            user_candidates[names] = 1
            tweets_candidates[names] = [[tweet1_text],[tweet2_text]]
        else:
            if tweet1_text not in tweets_candidates[names][0] and tweet1_text not in tweets_candidates[names][1] and tweet2_text not in tweets_candidates[names][0] and tweet2_text not in tweets_candidates[names][1]:
                user_candidates[names] += 1
                tweets_candidates[names][0].append(tweet1_text)
                tweets_candidates[names][1].append(tweet2_text)

100%|██████████| 416395/416395 [01:59<00:00, 3486.74it/s]


In [58]:
print(len(user_candidates))

279233


In [61]:
user_matches = [i for i in user_candidates if user_candidates[i] >= user_thereshold]

for i in user_matches:
    if user_candidates[i] >= user_thereshold:
        print(f"Evaluando usuarios {i}, con {user_candidates[i]} tweets similares")
        for j in range(user_candidates[i]):
            print(f"Tweet {j + 1} del Usuario: {i[0]}")
            print(tweets_candidates[i][0][j])
            print(f"Tweet {j + 1} del Usuario: {i[1]}")
            print(tweets_candidates[i][1][j])
            if j == 1:
                break
        print("-----------------------------------------")
        if i == 2:
            break

Evaluando usuarios ('JumpingStone20', 'v1_elena'), con 4 tweets similares
Tweet 1 del Usuario: JumpingStone20
rechazada esta indicación: "el estado no puede en caso alguno expropiar, confiscar o nacionalizar los ahorros de los trabajad…
Tweet 1 del Usuario: v1_elena
207.rechazada: “el estado no puede en caso alguno expropiar, confiscar o nacionalizar los ahorros de los trabajadores”.…
Tweet 2 del Usuario: JumpingStone20
#rechazodesalida
Tweet 2 del Usuario: v1_elena
#rechazodesalida2022
-----------------------------------------
Evaluando usuarios ('canaito031', 'gloria_patriota'), con 5 tweets similares
Tweet 1 del Usuario: canaito031
#rechazocrece
Tweet 1 del Usuario: gloria_patriota
#rechazocrece 🇨🇱🇨🇱
Tweet 2 del Usuario: canaito031
#rechazoelmamarrachocomunista
Tweet 2 del Usuario: gloria_patriota
#asíno #rechazoelmamarrachocomunista
-----------------------------------------
Evaluando usuarios ('canaito031', 'marg9173'), con 4 tweets similares
Tweet 1 del Usuario: canaito031
#rechaz

In [62]:
print(len(user_matches))
user_matches

31


[('JumpingStone20', 'v1_elena'),
 ('canaito031', 'gloria_patriota'),
 ('canaito031', 'marg9173'),
 ('Alexia19542695', 'BurguesProleta'),
 ('ProgreB', 'canaito031'),
 ('Alexia19542695', 'JumpingStone20'),
 ('JumpingStone20', 'canaito031'),
 ('JumpingStone20', 'janelg88'),
 ('AnitaVF4', 'JumpingStone20'),
 ('JumpingStone20', 'silvanafati77'),
 ('JumpingStone20', 'andresjavierCL'),
 ('JumpingStone20', 'patiisep'),
 ('ginniasa', 'giovannaroa'),
 ('Giovann56636968', 'ygorayeb'),
 ('AbrahamUrnas', 'clauriv56002711'),
 ('DosHablo', 'eram_pao'),
 ('BurguesProleta', 'canaito031'),
 ('BonitaClauditaT', 'janelg88'),
 ('Dpm_Chile2020', 'Maurici07154782'),
 ('BurguesProleta', 'JumpingStone20'),
 ('Giovann56636968', 'berfontaine'),
 ('JumpingStone20', 'marg9173'),
 ('andresjavierCL', 'marg9173'),
 ('janelg88', 'silvanafati77'),
 ('Miltonterasss', 'marg9173'),
 ('JumpingStone20', 'bybleanstartup'),
 ('ProgreB', 'janelg88'),
 ('AnitaVF4', 'marg9173'),
 ('Miltonterasss', 'silvanafati77'),
 ('Alexia1954

## Referencias

Teoría: http://infolab.stanford.edu/~ullman/mmds/ch3.pdf

Código referenciado: https://github.com/nicoDs96/Document-Similarity-using-Python-and-PySpark/blob/master/LSH/DM_HW2_Ex2.ipynb